In [1]:
from pprint import pprint
from dataclasses import asdict
import datetime as dt
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from itables import show

from martignac.nomad.uploads import get_upload_by_id, get_all_my_uploads
from martignac.nomad.datasets import get_dataset_by_id
from martignac.nomad.users import get_user_by_id
from martignac.nomad.entries import (
    query_entries, 
    get_entry_by_id, 
    get_entries_of_upload, 
    find_entries_corresponding_to_job, 
    download_raw_data_of_job,
    get_entries_of_my_uploads,
    get_entries_in_database,
    DEFAULT_DATABASE,
    DEFAULT_USE_PROD
)

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">link</a>'.format(val)

In [2]:
# entries
entries = get_entries_in_database()
df_e = pd.json_normalize({**asdict(e), "url": e.nomad_gui_url} for e in entries)
df_e = df_e[["entry_id", "upload_id", "entry_type", "entry_create_time", "url"]]


# uploads
upload_entries = get_entries_of_my_uploads()
df_u = pd.json_normalize(
    {**asdict(e), "workflow_name": e.workflow_name, "state_point": e.state_point, "mdp_files": e.mdp_files, "url": e.nomad_gui_url} 
    for e in upload_entries if e.comment and e._comment_dict.get("state_point") and e._comment_dict.get("workflow_name")
)
sp_columns = [c for c in df_u.columns if "state_point" in c]
df_u = df_u[
    ["entry_id", "upload_id", "entry_name", "entry_type", "workflow_name", "mdp_files", *sp_columns, "entry_create_time", "url"]
]
df_u["entry_create_time"] = pd.to_datetime(df_u["entry_create_time"]).dt.date

INFO:martignac.nomad.utils:Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query


INFO:martignac.nomad.entries:retrieving entry IWfLp8VCyT7z9t3BtVy21Q5WZSRW on test server


INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/IWfLp8VCyT7z9t3BtVy21Q5WZSRW


INFO:martignac.nomad.users:retrieving user 30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b on prod server


INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/users/30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b


...

INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/VRlAv-gdQBKxU9CrSU8T2w/entries


INFO:martignac.nomad.entries:retrieving entries for upload J-08brjXSZC5JBTOAfQLJA on test server


INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/J-08brjXSZC5JBTOAfQLJA/entries


INFO:martignac.nomad.entries:retrieving entries for upload DjbzajfaToyB18CQR44ywQ on test server


INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/DjbzajfaToyB18CQR44ywQ/entries


In [3]:
#| content: valuebox
#| title: "Number of entries"
#| icon: plus-circle
#| color: "success"
dict(
  value = len(df_e) + len(df_u)
)

{'value': 17}

In [4]:
#| content: valuebox
#| title: "Dataset ID"
dict(
    value = DEFAULT_DATABASE,
    color = "secondary"
)

{'value': 'HJdEI1q4SV-c5Di43BTT_Q', 'color': 'secondary'}

In [5]:
#| title: "Entries"
show(df_e.style.format({'url': make_clickable}))

entry_id,upload_id,entry_type,entry_create_time,url
IWfLp8VCyT7z9t3BtVy21Q5WZSRW,Hbwh9S0-RLehQIh90nfhAQ,GROMACS MolecularDynamics,2024-02-14 09:43:59.527000+00:00,link


In [6]:
#| title: "Uploads"
show(df_u.style.format({'url': make_clickable}))

entry_id,upload_id,entry_name,entry_type,workflow_name,mdp_files,state_point.type,state_point.solute_name,state_point.solvent_name,entry_create_time,url
prsfcogv6852DaQcPdGzh0OqyCER,VRlAv-gdQBKxU9CrSU8T2w,solute_generation.archive.yaml,Workflow,SoluteGenFlow,8f3517435fd911f25f11440d8eb22caa,solute,P6,nan,2024-03-19,link
LojhthjtotmEKo-WrOvbB_yQEmcj,VRlAv-gdQBKxU9CrSU8T2w,GROMACS GeometryOptimization simulation,GROMACS GeometryOptimization,SoluteGenFlow,8f3517435fd911f25f11440d8eb22caa,solute,P6,nan,2024-03-19,link
7Tik7MPpFHYJ1l5DskIL7HtPbXZx,J-08brjXSZC5JBTOAfQLJA,GROMACS MolecularDynamics simulation,GROMACS MolecularDynamics,SolventGenFlow,ba2c4618058c83ba0ab2d9d7ba76bd1f,solvent,nan,HD,2024-03-19,link
Umudk_qAcSEBx932rJl5R9Uhar8D,J-08brjXSZC5JBTOAfQLJA,solvent_generation.archive.yaml,Workflow,SolventGenFlow,ba2c4618058c83ba0ab2d9d7ba76bd1f,solvent,nan,HD,2024-03-19,link
0M435ZEJ8-mzHL9LEy8ONHQothr_,J-08brjXSZC5JBTOAfQLJA,GROMACS GeometryOptimization simulation,GROMACS GeometryOptimization,SolventGenFlow,ba2c4618058c83ba0ab2d9d7ba76bd1f,solvent,nan,HD,2024-03-19,link
M-e-GzQtS806eSVYmaBX_CwMdlxC,J-08brjXSZC5JBTOAfQLJA,GROMACS MolecularDynamics simulation,GROMACS MolecularDynamics,SolventGenFlow,ba2c4618058c83ba0ab2d9d7ba76bd1f,solvent,nan,HD,2024-03-19,link
ekOzn6S0BkV6CLtryv2BDfNM5uXG,DjbzajfaToyB18CQR44ywQ,full_workflow.solute_solvation.archive.yaml,Workflow,SoluteSolvationFlow,8f3517435fd911f25f11440d8eb22caa,solute_solvation,P6,HD,2024-03-19,link
Rm6Fu2W67OLX4i4cIy_wkGarEmb1,DjbzajfaToyB18CQR44ywQ,solute_generation.archive.yaml,Workflow,SoluteSolvationFlow,8f3517435fd911f25f11440d8eb22caa,solute_solvation,P6,HD,2024-03-19,link
rLjgLzzh4Sm-cuUi8yoebZGjkNRt,DjbzajfaToyB18CQR44ywQ,GROMACS GeometryOptimization simulation,GROMACS GeometryOptimization,SoluteSolvationFlow,8f3517435fd911f25f11440d8eb22caa,solute_solvation,P6,HD,2024-03-19,link
btV2NAWFGhHh5fZMaUlXqOZwvosP,DjbzajfaToyB18CQR44ywQ,solute_solvation.archive.yaml,Workflow,SoluteSolvationFlow,8f3517435fd911f25f11440d8eb22caa,solute_solvation,P6,HD,2024-03-19,link


In [7]:
#| title: "Uploads"
show(
    df_u[(df_u["workflow_name"] == "SoluteGenFlow") & (df_u["entry_type"] == "Workflow")]
    .drop(["workflow_name", "state_point.type", "state_point.solvent_name", 
           # "state_point.lambda_state"
          ], axis=1)
    .sort_values("state_point.solute_name")
    .style.format({'url': make_clickable})
)

,entry_id,upload_id,entry_name,entry_type,mdp_files,state_point.solute_name,entry_create_time,url
0,prsfcogv6852DaQcPdGzh0OqyCER,VRlAv-gdQBKxU9CrSU8T2w,solute_generation.archive.yaml,Workflow,8f3517435fd911f25f11440d8eb22caa,P6,2024-03-19,link


In [8]:
#| title: "Uploads"
show(
    df_u[(df_u["workflow_name"] == "SolventGenFlow") & (df_u["entry_type"] == "Workflow")]
    .drop(["workflow_name", "state_point.type", "state_point.solute_name", 
           # "state_point.lambda_state"
          ], axis=1)
    .sort_values("state_point.solvent_name")
    .style.format({'url': make_clickable})
)

,entry_id,upload_id,entry_name,entry_type,mdp_files,state_point.solvent_name,entry_create_time,url
3,Umudk_qAcSEBx932rJl5R9Uhar8D,J-08brjXSZC5JBTOAfQLJA,solvent_generation.archive.yaml,Workflow,ba2c4618058c83ba0ab2d9d7ba76bd1f,HD,2024-03-19,link


In [9]:
#| title: "Uploads"
show(
    df_u[(df_u["workflow_name"] == "SoluteSolvationFlow") & (df_u["entry_type"] == "Workflow") & (df_u["entry_name"] == "full_workflow.solute_solvation.archive.yaml")]
    .drop(["entry_name", "entry_type", "workflow_name", "state_point.type", 
           # "state_point.lambda_state"
          ], axis=1)
    .sort_values("state_point.solvent_name")
    .style.format({'url': make_clickable})
)

,entry_id,upload_id,mdp_files,state_point.solute_name,state_point.solvent_name,entry_create_time,url
6,ekOzn6S0BkV6CLtryv2BDfNM5uXG,DjbzajfaToyB18CQR44ywQ,8f3517435fd911f25f11440d8eb22caa,P6,HD,2024-03-19,link


In [10]:
#| title: "Uploads"
show(
    df_u[(df_u["workflow_name"] == "AlchemicalTransformationFlow") & (df_u["entry_type"] == "Workflow") & (df_u["entry_name"] == "full_workflow.alchemical_transformation.archive.yaml")]
    .drop(["entry_name", "entry_type", "workflow_name", "state_point.type", 
           # "state_point.lambda_state"
          ], axis=1)
    .sort_values("state_point.solvent_name")
    .style.format({'url': make_clickable})
)

,entry_id,upload_id,mdp_files,state_point.solute_name,state_point.solvent_name,entry_create_time,url
